# Liquidity pools

In [ ]:
#| default_exp pool

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from dataclasses import dataclass
from typing import Tuple, Dict, Optional
from sugar.token import Token
from sugar.helpers import normalize_address

In [ ]:
#| export 

@dataclass(frozen=True)
class Price:
    """Data class for Token Price

    based on:
    https://github.com/velodrome-finance/oracle/blob/main/contracts/VeloOracle.sol
    """

    token: Token
    price: float

    @property
    def pretty_price(self) -> float: return round(self.price, 5)

In [ ]:
#| export

@dataclass(frozen=True)
class Amount:
    token: Token
    amount: float
    price: "Price"

    @classmethod
    def build(
        cls,
        address: str,
        amount: float,
        tokens: Dict[str, Token],
        prices: Dict[str, "Price"],
    ) -> "Amount":
        address = normalize_address(address)

        if address not in tokens or address not in prices:
            return None

        token = tokens[address]

        return Amount(
            token=token, amount=token.value_from_bigint(amount), price=prices[address]
        )

    @property
    def amount_in_stable(self) -> float:
        return self.amount * self.price.price


In [ ]:
#| export

@dataclass(frozen=True)
class LiquidityPool:
    """Data class for Liquidity Pool

    based on:
    https://github.com/velodrome-finance/sugar/blob/v2/contracts/LpSugar.vy#L31
    """

    lp: str
    factory: str
    symbol: str
    is_stable: bool
    # concentrated liquidity pools
    is_cl: bool
    total_supply: float
    decimals: int
    token0: Token
    #reserve0: Amount
    token1: Token
    #reserve1: Amount
    #token0_fees: Amount
    #token1_fees: Amount
    pool_fee: float
    gauge_total_supply: float
    #emissions: Amount
    emissions_token: Token
    #weekly_emissions: Amount
    nfpm: str
    alm: str

    @classmethod
    def from_tuple(
        cls, t: Tuple, tokens: Dict[str, Token] #,prices: Dict[str, Price]
    ) -> Optional["LiquidityPool"]:
        token0, token1, pool_type = normalize_address(t[7]), normalize_address(t[10]), t[4]
        # token0_fees = t[23]
        # token1_fees = t[24]
        emissions_token = normalize_address(t[20])
        # emissions = t[19]

        # seconds_in_a_week = 7 * 24 * 60 * 60

        # Sugar.all returns a tuple with the following structure:
        # { "name": "lp", "type": "address" },          <== 0
        # { "name": "symbol", "type": "string" },       <== 1
        # { "name": "decimals", "type": "uint8" },      <== 2
        # { "name": "liquidity", "type": "uint256" },   <== 3
        # { "name": "type", "type": "int24" },          <== 4
        # { "name": "tick", "type": "int24" },          <== 5
        # { "name": "sqrt_ratio", "type": "uint160" },  <== 6
        # { "name": "token0", "type": "address" },      <== 7
        # { "name": "reserve0", "type": "uint256" },    <== 8
        # { "name": "staked0", "type": "uint256" },     <== 9
        # { "name": "token1", "type": "address" },      <== 10
        # { "name": "reserve1", "type": "uint256" },    <== 11
        # { "name": "staked1", "type": "uint256" },     <== 12
        # { "name": "gauge", "type": "address" },        <== 13
        # { "name": "gauge_liquidity", "type": "uint256" },  <== 14
        # { "name": "gauge_alive", "type": "bool" },        <== 15
        # { "name": "fee", "type": "address" },             <== 16
        # { "name": "bribe", "type": "address" },           <== 17
        # { "name": "factory", "type": "address" },         <== 18
        # { "name": "emissions", "type": "uint256" },       <== 19
        # { "name": "emissions_token", "type": "address" },  <== 20
        # { "name": "pool_fee", "type": "uint256" },        <== 21
        # { "name": "unstaked_fee", "type": "uint256" },    <== 22
        # { "name": "token0_fees", "type": "uint256" },     <== 23
        # { "name": "token1_fees", "type": "uint256" },    <== 24
        # { "name": "nfpm", "type": "address" },           <== 25
        # { "name": "alm", "type": "address" }             <== 26

        token0, token1 = tokens.get(token0), tokens.get(token1)
        if not token0 or not token1: return None
        symbol = f"CL{pool_type}-{token0.symbol}/{token1.symbol}" if pool_type > 0  else f"{'s' if pool_type == 0 else 'v'}AMM-{token0.symbol}/{token1.symbol}"
    
        return LiquidityPool(
            lp=normalize_address(t[0]),
            factory=normalize_address(t[18]),
            symbol=symbol,
            # stable pools have type set to 0
            is_stable=pool_type == 0,
            is_cl=pool_type > 0,
            total_supply=t[3],
            decimals=t[2],
            token0=token0,
            #reserve0=Amount.build(token0, t[8], tokens, prices),
            token1=token1,
            #reserve1=Amount.build(token1, t[11], tokens, prices),
            #token0_fees=Amount.build(token0, token0_fees, tokens, prices),
            #token1_fees=Amount.build(token1, token1_fees, tokens, prices),
            pool_fee=t[21],
            gauge_total_supply=t[14],
            emissions_token=tokens.get(emissions_token),
            #emissions=Amount.build(emissions_token, emissions, tokens, prices),
            # weekly_emissions=Amount.build(
            #     emissions_token, emissions * seconds_in_a_week, tokens, prices
            # ),
            nfpm=normalize_address(t[25]),
            alm=normalize_address(t[26]),
        )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()